# Linking PG&E utility data with building polygons
This notebook will take Global Building Atlas polygons and link to the nearest PG&E feeder line.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import box
from shapely.geometry import MultiLineString
import matplotlib.pyplot as plt
import glob
import os
import folium

In [2]:
pd.set_option('display.max_columns', None)

# 1a. DERCiruits.shp

In [3]:
# Load data
pge_der_circuits = gpd.read_file("../../../../capstone/electrigrid/data/utilities/pge_shapefiles/DERCircuits.shp")
pge_der_circuits.head()

,circuitid,circuitnam,subname,aafscoolin,aafscool_1,aafscool_2,aafscool_3,aafscool_4,aafsheatin,aafsheat_1,aafsheat_2,aafsheat_3,aafsheat_4,aggregatea,aggregat_1,aggregatee,aggregat_2,aggregat_3,aggregat_4,aggregat_5,aggregat_6,aggregat_7,aggregat_8,aggregat_9,aggregaten,aggrega_10,aggrega_11,aggrega_12,aggregatep,bank_id,bank_name,distributi,division,energyeffi,energyef_1,energyef_2,energyef_3,energyef_4,energystor,energyst_1,energyst_2,energyst_3,energyst_4,energyst_5,energyst_6,energyst_7,energyst_8,energyst_9,evfleetmw2,evfleetm_1,evfleetm_2,evfleetm_3,evfleetm_4,evpublicl2,evpublic_1,evpublic_2,evpublic_3,evpublic_4,evresl1mw2,evresl1m_1,evresl1m_2,evresl1m_3,evresl1m_4,evresl2mw2,evresl2m_1,evresl2m_2,evresl2m_3,evresl2m_4,evurbandcf,evurband_1,evurband_2,evurband_3,evurband_4,evworkplac,evworkpl_1,evworkpl_2,evworkpl_3,evworkpl_4,facility_i,facility_n,feeder_id,feeder_nam,newagricul,newagric_1,newagric_2,newagric_3,newagric_4,newcommerc,newcomme_1,newcomme_2,newcomme_3,newcomme_4,newindustr,newindus_1,newindus_2,newindus_3,newindus_4,newresiden,newresid_1,newresid_2,newresid_3,newresid_4,last_updat,photovolta,photovol_1,photovol_2,photovol_3,photovol_4,SHAPE_Leng,geometry
0,012011101,OAKLAND C 1101,OAKLAND C,0.01,0.02,0.02,0.03,0.05,0.00,0.01,0.02,0.03,0.05,0.12,0.11,-0.61,-0.06,0.00,0.00,0.02,0.00,0.00,0.00,0.24,0.00,0.00,0.00,0.00,-0.82,012011251,NETWORK GROUP OAKLAND C 1251,Bay Area,East Bay,-0.07,-0.10,-0.13,-0.15,-0.15,-0.01,-0.01,-0.01,-0.01,-0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.24,12011101,OAKLAND C 1101,012011101,OAKLAND C 1101,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2025-09-12,-0.04,-0.11,-0.16,-0.22,-0.29,4033.818754,"MULTILINESTRING ((564059.182 4184096.280, 5640..."
1,012011102,OAKLAND C 1102,OAKLAND C,0.01,0.02,0.04,0.05,0.06,0.00,0.01,0.02,0.05,0.08,0.18,0.16,-0.46,-0.02,-0.01,0.00,0.02,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,-0.73,012011251,NETWORK GROUP OAKLAND C 1251,Bay Area,East Bay,-0.05,-0.07,-0.11,-0.11,-0.12,0.00,0.00,0.00,-0.01,-0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,12011102,OAKLAND C 1102,012011102,OAKLAND C 1102,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2025-09-12,-0.05,-0.09,-0.14,-0.21,-0.24,4937.485621,"MULTILINESTRING ((564143.703 4184260.180, 5641..."
2,012011103,OAKLAND C 1103,OAKLAND C,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,012011251,NETWORK GROUP OAKLAND C 1251,Bay Area,East Bay,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,12011103,OAKLAND C 1103,012011103,OAKLAND C 1103,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,2025-09-12,-99.99,-99.99,-99.99,-99.99,-99.99,3925.468328,"MULTILINESTRING ((564081.619 4184244.407, 5640..."
3,012011104,OAKLAND C 1104,OAKLAND C,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,012011251,NETWORK GROUP OAKLAND C 1251,Bay Area,East Bay,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99.99,-99